In [284]:
import pandas as pd

%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt

import features as fea
import data_source as ds
reload(ds)
reload(fea)

<module 'features' from 'features.py'>

In [271]:
dt_df = ds.get_date_df()
obj_df = ds.get_obj_df()

train_enroll = ds.get_enrollment_df('train')
train_log = ds.get_log_df('train')
train_labels = ds.get_labels_df()

test_enroll = ds.get_enrollment_df('test')
test_log = ds.get_log_df('test')

sample_sub_df = ds.get_sample_submission()

In [286]:
train_enroll.shape

(120542, 3)

In [287]:
"""
Base Feature
"""
train_base = fea.activity_total_and_percentile(enroll_df=train_enroll, log_df=train_log, dt_df=dt_df).sort('enrollment_id')
print train_base.shape

(120542, 18)


In [288]:
"""
Course Level Feature
"""
new_train_base = pd.merge(train_base, fea.avg_activity_per_course_event(enroll_df=train_enroll, log_df=train_log), on=['course_id'], how='left')
print new_train_base.shape

(120542, 26)


In [289]:
"""
Day Range Level Feature
"""
whole_new_train_base = pd.merge(new_train_base, fea.activity_filtered(enroll_df=train_enroll, log_df=train_log, dt_df=dt_df), on=['enrollment_id'], how='left')
print whole_new_train_base.shape

(120542, 35)


In [290]:
whole_new_train_base.head()

,enrollment_id,course_id,prctl_total_activity,total_activity,total_navigate_activity,prctl_navigate_activity,total_access_activity,prctl_access_activity,total_problem_activity,prctl_problem_activity,...,avg_activity_wiki,total_activity_top_5,total_activity_tail_5,total_activity_diff_5,total_activity_top_10,total_activity_tail_10,total_activity_diff_10,total_activity_top_15,total_activity_tail_15,total_activity_diff_15
0,1,DPnLzkJJqOOPRJfBxIHbQEERiYHu5ila,0.947588,314,25,0.872312,107,0.910676,87,0.994042,...,1.889215,13,55,42,57,185,128,129,252,123
1,3,7GRhBDsirIGkRZBtSMEzNTyDr2JQm4xx,0.881161,288,14,0.847738,79,0.845375,138,0.937542,...,2.228070,31,93,62,61,105,44,183,217,34
2,4,DPnLzkJJqOOPRJfBxIHbQEERiYHu5ila,0.707227,99,15,0.743703,64,0.791174,6,0.667748,...,1.889215,31,0,-31,43,23,-20,76,47,-29
3,5,7GRhBDsirIGkRZBtSMEzNTyDr2JQm4xx,0.968940,633,30,0.936529,226,0.957461,170,0.963201,...,2.228070,147,93,-54,179,222,43,411,235,-176
4,6,AXUJZGmZ0xaYSWazu8RQ1G5c76ECT1Kd,0.627717,23,5,0.620514,12,0.673569,2,0.745531,...,3.573103,0,23,23,0,23,23,0,23,23


In [297]:
whole_new_train_base.columns.tolist()
features = [
 #'course_id',
 'prctl_total_activity',
 'total_activity',
 'total_navigate_activity',
 'prctl_navigate_activity',
 'total_access_activity',
 'prctl_access_activity',
 'total_problem_activity',
 'prctl_problem_activity',
 'total_page_close_activity',
 'prctl_page_close_activity',
 'total_video_activity',
 'prctl_video_activity',
 'total_discussion_activity',
 'prctl_discussion_activity',
 'total_wiki_activity',
 'prctl_wiki_activity',
 'avg_activity',
 'avg_activity_navigate',
 'avg_activity_access',
 'avg_activity_problem',
 'avg_activity_page_close',
 'avg_activity_video',
 'avg_activity_discussion',
 'avg_activity_wiki',
 'total_activity_top_5',
 'total_activity_tail_5',
 'total_activity_diff_5',
 'total_activity_top_10',
 'total_activity_tail_10',
 'total_activity_diff_10',
 'total_activity_top_15',
 'total_activity_tail_15',
 'total_activity_diff_15']
train_base = whole_new_train_base[features]
labels = train_labels[1]

In [ ]:
import sklearn.metrics as metrics
from sklearn import cross_validation
from sklearn import ensemble, preprocessing
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import cross_val_score
from sklearn import metrics
from sklearn.pipeline import Pipeline
# Random Forest Classifier
clf = ensemble.RandomForestClassifier(n_jobs=-1, n_estimators=1000, min_samples_split=1)
cv = cross_validation.ShuffleSplit(train_base.shape[0], n_iter=5,
        test_size=0.3, random_state=0)
scores = cross_validation.cross_val_score(
            clf, train_base, labels, cv=cv)
print("Accuracy: %0.5f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

clf = ensemble.RandomForestClassifier(n_jobs=-1, n_estimators=1000, min_samples_split=1)
cv = cross_validation.ShuffleSplit(train_base.shape[0], n_iter=10,
        test_size=0.3, random_state=0)
scores = cross_validation.cross_val_score(
            clf, train_base, labels, cv=cv, scoring='roc_auc')
print("ROC: %0.5f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

In [ ]:
clf.fit(train_base, labels)
predicted_labels = clf.predict(train_base)

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
fpr = dict()
tpr = dict()
roc_score = roc_auc_score(labels, predicted_labels)
fpr['model'], tpr['model'], _ = roc_curve(labels, predicted_labels)

plt.figure()
plt.plot(fpr['model'], tpr['model'], label='ROC curve (area = %0.5f)' % roc_score)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 0.1])
plt.ylim([0.1, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()